In [1]:
from neo4j import GraphDatabase
import re
advent = "Day1"
workdir = 'C:\\Users\\ed.deane\\OneDrive - Sabio Ltd\\Documents\\Customers\\IKEA\\Knowledge\\Python\\AdventOfCode\\2023\\'

f = open(workdir+advent+"\\test_input_part2.txt", "r")
listOfCalibrationValues = []
for line in f:
    line = line.strip()
    print (line)
    listOfCalibrationValues.append(line)

listOfCalibrationValues

two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen


['two1nine',
 'eightwothree',
 'abcone2threexyz',
 'xtwone3four',
 '4nineeightseven2',
 'zoneight234',
 '7pqrstsixteen']

In [2]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "advent_of_code"
# Cypher query to create a node with the index as the node name and value as the 'prop' property
query_del = "MATCH (n) DETACH DELETE n"

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    with driver.session() as session:
            session.run(query_del)

query = "CREATE (:line {name: $nodeName, calibration_value: $calibration_value})"

# Function to create nodes in Neo4j
def create_node(tx, nodeName, calibration_value):
    tx.run(query, nodeName=nodeName, calibration_value=calibration_value)

# Connect to the Neo4j database
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    # Iterate through the list and create nodes
    for index, value in enumerate(listOfCalibrationValues):
        # Use a transaction to create each node
        with driver.session() as session:
            session.write_transaction(create_node, str(index), value)

C:\Users\ed.deane\AppData\Local\Temp\ipykernel_3184\170366737.py:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, str(index), value)


In [3]:
ints = [("one",1), ("two",2), ("three",3), ("four",4), ("five",5), ("six",6), ("seven",7), ("eight",8),("nine",9)]

query_num = "CREATE (:number {name: $nodeName, nodeValue: $nodeValue})"

#add each int as a node 
with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    # Use a transaction to execute the query
    for i in ints:
        with driver.session() as session:
            session.run(query_num, nodeName=i[0], nodeValue=i[1])
        


In [4]:
def findOccurrences(s, ch): 
    indexes = [x.start() for x in re.finditer(ch, s)]
    return indexes

# link up each line to the first and last numbers on it
def get_all_numbers(cal_val):
    # loop through each int and get each of its positions in the string
    all_numbers = []
    for i in ints:
        all_ints = findOccurrences(cal_val,str(i[1]))
        for j in all_ints:            
            all_numbers.append((j,i[1]))
        
        #repeat for the strings
        all_ints = findOccurrences(cal_val,i[0])
        for j in all_ints:            
            all_numbers.append((j,i[1]))
    
    # Put the results into a list where the position is first and the int is second
    return (all_numbers)

def get_first_last(st):
    
    all_nums_from_line = get_all_numbers(st)
    
    sorted_ls = sorted(all_nums_from_line, key=lambda x: x[0])
    
    if len(sorted_ls) == 0:
        ret = 0
    #elif len(sorted_ls) == 1:
    #    ret = sorted_ls[0][1]
    else:
        ret = str(sorted_ls[0][1])+str(sorted_ls[-1][1])
    
    return int(ret)

def get_num_in_words(num):
    for i in ints:
        if i[1] == num:
            return i[0]

def add_relationships (line, number, position):
    
    query_num = """
    MATCH (p:line {calibration_value: $nodeLine})
    MATCH (c:number {name: $nodeNumber})
    CREATE (p)-[:CONTAINS  {position:$position}]->(c)
    """

    #add each int as a node 
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
        with driver.session() as session:
            session.run(query_num, nodeLine=line, nodeNumber=number, position=position)




In [ ]:
sum(list(map(get_first_last, listOfCalibrationValues)))

In [20]:
for i in listOfCalibrationValues:
    first_last = str(get_first_last(i))
    print (i, first_last[0],first_last[1])
    add_relationships (i, first_last[0], 23)

two1nine 2 9


TypeError: get_num_in_words() missing 1 required positional argument: 'num'

'three'